# With Heuristic (MRV, LCV)

In [10]:
#A function that chooses the next node to be a node with the minimum remaining values.
#In our case a vertext with the least number of neighboors has the priority and is choosen.s
#To find the correct node the vertex is checked to be uncolored and the number of neighboors is compared with each others. 
def MRV_function(domain,is_colored):
    min = 1000
    node = -1
    for i in range(num_of_vertices):
        if (len(domain[i]) < min and is_colored[i] ==-1):
            min = len(domain[i])
            node = i
    return node
    
#A function that chooses the color which contraints the color choice of the neighbooring vertices the least.
#The color from the domain of the specific vertex is checked with the neighbooring and the color with the min constraints is found.
#then the values are sorted.
def LCV_function(v,adjList,domain):
    sorted_vals=[]
    for c in domain[v]:
        min=1000
        for i in adjList[v]:
            ar = len(domain[i])
            if c in domain[i]:
                ar = ar-1
            if(ar<min):
                min = ar
        sorted_vals.append([c,min])
        s = sorted(sorted_vals, key=lambda x: x[1],reverse=True)
        color = [val[0] for val in s]
    return color

#A function that checks the safety condition for both backtracking with heuristic and pure backtracking functions
#If color to be assigned is already assigned to a neighbooring vertex then it is False. s
def ifSafe(v,i,colors_ind,adjList):

    for j in adjList[v]:
        if i ==colors_ind[j]:
            return 0

    return 1

#Backtracking with heuristic combines in itself both the MRV and MCV heuristics. 
#Which means that the next node to be chosen satisfies MRV and the color to be assigned to the 
#vertex satisfies LCVs. The rest parts of the function are almost the same with pure backtracking funciton.
def backtrack_with_heuristic(colors_ind,adjList,domain,is_colored):
    if is_colored.count(-1)==0:
        finish=time.time()
        return colors_ind
    next_v = MRV_function(domain, is_colored)
    c_value = LCV_function(next_v,adjList,domain)
    for c in c_value: 
      if (ifSafe(next_v, c, colors_ind, adjList))==True:
        colors_ind[next_v] = c
        is_colored[next_v] = 1
        result = backtrack_with_heuristic(colors_ind,adjList,domain,is_colored)
        if result!=0:
          return result
    colors_ind[next_v]=-1
    is_colored[next_v]=-1
    return 0  


# Pure Backtracking 

In [11]:
#A function to print out the final results of pure backtracking funciton
def printOut(colors_ind):
  output=[]
  for i in range(len(colors_ind)):
    output.append(list_of_colors[colors_ind[i]])
  print("\n Results:",output)

#A function that calculates the result with the pure backtracking algorithm.
#Once the node reaches the last vertex the calculation stops and results are printed out
#If node did not reach the final vertex yet then for each color in the domain of colors the node
#is checked for safety (is the neighbooring node has the same color). If all fine the proceeds with 
#calling the backtracking function again. If failure occurs then resets the value to -1.
def backtrack(v, colors_ind,  adjList):
    start = time.time()
    if v == num_of_vertices:
        printOut(colors_ind)
        finish=time.time()
        print(f"\nTime: {round((finish - start),5)} seconds")     
        return 1
    else:
        for c in range(0, num_of_colors):
            if (ifSafe(v, c, colors_ind, adjList)):
                colors_ind[v] = c
                if (backtrack(v + 1, colors_ind,  adjList) == 1):
                    return 1
            colors_ind[v] = -1

        return 0


# Initialization and Run

In [12]:
import time

# defining essential variables global to use across the program
global num_of_vertices,num_of_edges,num_of_colors
edges=[]
values=[]
#reading the while extracting the core data to be processed further
with open("Colors.txt",'r') as vertices:
  for line in vertices.readlines(): 
    if not line.startswith("#"):
      if line.startswith("colors"):
        num = int(line.split('=')[1])
      else:
        #storing edges and vertices separately as we will need them booth across the programm
        s = int(line.split(',')[0]) 
        d = int(line.split(',')[1]) 
        edges.append((s,d))
        values.append(s)
        values.append(d)
#defining the core variables of the program
num_of_vertices = len(set(values))+1
num_of_edges = len(edges)
num_of_colors = num

#creating the list of 10 colors that can be chosen and assign to vertices across the program
list_of_colors = ["BLUE", "GREEN", "RED", "YELLOW", "ORANGE", "PURPLE",
            "BLACK", "VIOLET", "WHITE", "TIFFANY_BLUE"]
#both colors_ind and is_colored are filled with value of -1 as much as we have vertices in our file
#one stores the indices of the colors the other stores which vertices are colored and which not
colors_ind = [-1 for i in range(num_of_vertices)]
is_colored = [-1 for i in range(num_of_vertices)]

#from the list of edges the adjacency list is created.
#Namely the index represents the vertex and the values in the list represent the neighboors of that vertex
adjList = [[] for j in range(num_of_vertices)]
for (v_from, v_to) in edges:
  adjList[v_from].append(v_to)
  adjList[v_to].append(v_from)

#transform the adjacency list to one univeral list of lists(neighboors)
for i in range(num_of_vertices):
  f = list(set(adjList[i]))
  adjList[i] = f

#domain variable stores the list of colors per each index. The number of colors is defined in the document.
domain = [[] for k in range(num_of_vertices)]
for i in range(0,num_of_vertices):
  for j in range(0,num_of_colors):
    domain[i].append(j)

#Running the core functions and printing out the necessary information along with the time spent and color values assigned
print("....MVR LCV....")
print("\n Number of vertices:", num_of_vertices)
print("\n Number of edges:", num_of_edges)
print("\n Number of colors used:", num_of_colors)
start_1=time.time()
#backtracking with heuristic
res = backtrack_with_heuristic( colors_ind, adjList, domain,is_colored)
if res==0:
  print("\nNo Solution found! Try more colors.")
else:
  finish_1=time.time()
  print(f"\nTime: {round((finish_1 - start_1),5)} seconds")
  output1=[]
  for i in range(len(res)):
    output1.append(list_of_colors[res[i]])
  print("\n Results:",output1)

#pure backtracking
print("\n....Backtracking....")
print("\n Number of vertices:", num_of_vertices)
print("\n Number of edges:", num_of_edges)
print("\n Number of colors used:", num_of_colors)
output2 = backtrack(0,colors_ind,adjList)
if output2 ==0:
  print("\nNo Solution found! Try more colors.")


....MVR LCV....

 Number of vertices: 8

 Number of edges: 17

 Number of colors used: 4

Time: 0.00025 seconds

 Results: ['BLUE', 'BLUE', 'GREEN', 'RED', 'RED', 'GREEN', 'BLUE', 'YELLOW']

....Backtracking....

 Number of vertices: 8

 Number of edges: 17

 Number of colors used: 4

 Results: ['BLUE', 'BLUE', 'GREEN', 'RED', 'RED', 'GREEN', 'BLUE', 'YELLOW']

Time: 0.00029 seconds
